# Intercomparison of cross validation points

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_folder = 'data/clean/'

## Open first dataframe

In [3]:
dfs=[]
for f in os.listdir(data_folder):
    name, ext = os.path.splitext(f)
    if ext == '.csv':
        dfs.append(f)

In [4]:
df = pd.read_csv(data_folder+dfs[0]).set_index('Unnamed: 0').drop(['class', 'plot_id'], axis=1)
len(df)

211

## Join other dataframes where lat,lon and month match

Usin 'outer' method so we retain all observations

In [5]:
for f in dfs[1:]:
    other=pd.read_csv(data_folder+f).set_index('Unnamed: 0').drop(['class', 'plot_id'], axis=1)
    df = df.merge(other, on=['lat', 'lon', 'month'], suffixes=[None, f[0:3]], how='outer')

## Find unique mode for each lat,lon,month

In [6]:
u = df.filter(like='waterflag').mode(axis=1, dropna=False)
if len(u.columns) > 1:
    u = u.iloc[:, 0].where(u.iloc[:, 1:].isna().all(axis=1))

df['waterflag_mode'] = u

## Find counts

In [7]:
df['waterflag_mode'].value_counts()

1.0    273
0.0    121
2.0     18
Name: waterflag_mode, dtype: int64

In [8]:
df

,lat,lon,month,waterflag,waterflagAFR,waterflagAFR,waterflagAGR,waterflagAGR,waterflagRCM,waterflagRCM,waterflagAFR,waterflagAFR,waterflag_mode
0,7.103984,13.880937,1,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0
1,7.103984,13.880937,2,0.0,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN
2,7.103984,13.880937,4,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0
3,7.103984,13.880937,9,0.0,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN
4,7.103984,13.880937,10,0.0,0.0,0.0,2.0,2.0,NaN,0.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,5.999222,0.994027,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1023,5.999222,0.994027,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1024,5.999222,0.994027,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1025,5.999222,0.994027,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
